In [2]:
import cv2

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,800)

width = 0
height = 0

ret, frame1 = cap.read()
ret, frame2 = cap.read()

feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)



while cap.isOpened():
    p1, st, err = cv2.calcOpticalFlowPyrLK(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY), p0, None, **lk_params)

    if p1.shape[0] < 2:
        p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)
        width = 0
        height = 0
        p1, st, err = cv2.calcOpticalFlowPyrLK(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY), p0, None, **lk_params)

    new_corners = p1[st==1]
    old_corner = p0[st==1]
    
    width += new_corners[0][0] - old_corner[0][0]
    height += new_corners[0][1] - old_corner[0][1]
    

    if width >= 0:
        cv2.rectangle(frame1, (0, 0), (int(width), frame1.shape[0]), (0, 255, 0), -1)
    else:
        cv2.rectangle(frame1, (frame1.shape[1] - int(-width), 0), (frame1.shape[1], frame1.shape[0]), (0, 255, 0), -1)
    
    if height >= 0:
        cv2.rectangle(frame1, (0, 0), (frame1.shape[1], int(height)), (0, 255, 0), -1)
    else:
        cv2.rectangle(frame1, (0, frame1.shape[0] - int(-height)), (frame1.shape[1], frame1.shape[0]), (0, 255, 0), -1)
   
    cv2.imshow('Stabilized', frame1)                
    
    frame1 = frame2
    ret, frame2 = cap.read()
    p0 = new_corners.reshape(-1,1,2)
    
    key = cv2.waitKey(1)
    if key == 27:  
        break
    elif key == 114:
        p0 = cv2.goodFeaturesToTrack(cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY), mask = None, **feature_params)
        width = 0
        height = 0

cap.release()
cv2.destroyAllWindows()

In [1]:
from vidstab import VidStab

stabilizer = VidStab()
stabilizer.stabilize(input_path=0,
                     output_path='stable_webcam.avi',
                     max_frames=1000,
                     playback=True)